<a href="https://colab.research.google.com/github/PriyanshuRao-code/AI-Lab-Project/blob/main/team_24_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Don't do anything here. It is just a setup.
import os
import sys

repo_name = "AI-Lab-Project"
repo_url = "https://github.com/PriyanshuRao-code/AI-Lab-Project.git"
repo_path = f"/content/{repo_name}"

if os.path.exists(repo_path):
    print("Repository already exists at:", repo_path)
else:
    print("🚀 Cloning the repository...")
    os.system(f"git clone {repo_url}")

os.chdir(repo_path)
sys.path.append(repo_path)

print("Repository is ready to use at:", repo_path)

🚀 Cloning the repository...
Repository is ready to use at: /content/AI-Lab-Project


In [2]:
# Start your code from here

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.model_selection import train_test_split

In [4]:
original_dataframe = pd.read_csv('24.csv')

In [5]:
def remove_outliers_iqr(df_train, df_valid, df_test):

  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_train_non_numeric = df_train.select_dtypes(exclude=['number'])

  Q1 = df_train_numeric.quantile(0.25)
  Q3 = df_train_numeric.quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Removing outliers
  df_iqr = df_train_numeric[~((df_train_numeric < lower_bound) | (df_train_numeric > upper_bound)).any(axis=1)]
  df_train_cleaned = pd.concat([df_iqr, df_train_non_numeric.loc[df_iqr.index]], axis=1)

  return df_train_cleaned, df_valid, df_test

In [6]:
def remove_outliers_zscore(df_train, df_valid, df_test, z_score_threshold = 3):

  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_train_non_numeric = df_train.select_dtypes(exclude=['number'])

  z_scores = df_train_numeric.apply(zscore)
  df_z = df_train_numeric[(z_scores.abs() < z_score_threshold).all(axis=1)]  # Remove rows with Z-score >  z_score_threshold in any column
  df_train_cleaned = pd.concat([df_z, df_train_non_numeric.loc[df_z.index]], axis=1)

  return df_train_cleaned, df_valid, df_test

In [7]:
def remove_highly_correlated(df_train, df_valid, df_test, target_col="Hazardous", high_corr_threshold = 0.99):

  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])

  target_series = None
  if target_col in df_train_numeric.columns: # High correaltion with target column should not be dropped
    target_series = df_train[target_col]
    df_train_numeric = df_train_numeric.drop(columns=[target_col])

  high_corr_pairs = set()
  correlation_matrix = df_train_numeric.corr()

  for i in range(len(correlation_matrix.columns)):
    for j in range(i): # Lower triangular matrix
      if abs(correlation_matrix.iloc[i, j]) >= high_corr_threshold:
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        high_corr_pairs.add((col1, col2))

  columns_to_drop = {col2 for col1, col2 in high_corr_pairs}

  df_train_reduced = df_train.drop(columns=columns_to_drop)
  df_valid_reduced = df_valid.drop(columns=columns_to_drop)
  df_test_reduced = df_test.drop(columns=columns_to_drop)

  if target_series is not None: # If present in df_numeric and is removed, has to be added again
    df_train_reduced[target_col] = target_series

  return df_train_reduced, df_train_reduced, df_test_reduced

In [8]:
def normalize_minmax(df_train, df_test, df_valid, target_col="Hazardous"):
  scaler = MinMaxScaler()

  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_test_numeric = df_test.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_valid_numeric = df_valid.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])

  # Handling target column
  target_train, target_test, target_valid = None, None, None

  if target_col in df_train_numeric.columns:
    target_train = df_train[target_col]
    df_train_numeric = df_train_numeric.drop(columns=[target_col])

    target_test = df_test[target_col]
    df_test_numeric = df_test_numeric.drop(columns=[target_col])

    target_valid = df_valid[target_col]
    df_valid_numeric = df_valid_numeric.drop(columns=[target_col])

  # Normalization
  df_train[df_train_numeric.columns] = scaler.fit_transform(df_train_numeric)
  df_test[df_test_numeric.columns] = scaler.transform(df_test_numeric)
  df_valid[df_valid_numeric.columns] = scaler.transform(df_valid_numeric)

  # Again including the target_col
  if target_train is not None:
    df_train[target_col] = target_train
  if target_test is not None:
    df_test[target_col] = target_test
  if target_valid is not None:
    df_valid[target_col] = target_valid

  return df_train, df_test, df_valid

In [9]:
def standardize_data(df_train, df_test, df_valid, target_col="Hazardous"):
  scaler = StandardScaler()

  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_test_numeric = df_test.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])
  df_valid_numeric = df_valid.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])

  # Handling target column
  target_train, target_test, target_valid = None, None, None

  if target_col in df_train_numeric.columns:
    target_train = df_train[target_col]
    df_train_numeric = df_train_numeric.drop(columns=[target_col])

    target_test = df_test[target_col]
    df_test_numeric = df_test_numeric.drop(columns=[target_col])

    target_valid = df_valid[target_col]
    df_valid_numeric = df_valid_numeric.drop(columns=[target_col])

  # Standardization
  df_train[df_train_numeric.columns] = scaler.fit_transform(df_train_numeric)
  df_test[df_test_numeric.columns] = scaler.transform(df_test_numeric)
  df_valid[df_valid_numeric.columns] = scaler.transform(df_valid_numeric)


  # Again including the target_col
  if target_train is not None:
    df_train[target_col] = target_train
  if target_test is not None:
    df_test[target_col] = target_test
  if target_valid is not None:
    df_valid[target_col] = target_valid

  return df_train, df_test, df_valid

In [10]:
def label_encode_categorical(df_train, df_valid, df_test):

  categorical_cols = df_train.select_dtypes(include=['object']).columns
  label_encoders = {}


  for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df_train[col] = label_encoders[col].fit_transform(df_train[col])

    df_valid[col] = df_valid[col].map(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)
    df_valid[col] = df_valid[col].map(lambda x: label_encoders[col].transform([x])[0] if x in label_encoders[col].classes_ else -1)

  return df_train, df_valid, df_test

In [11]:
def select_high_corr_features(df_train, df_test, df_valid, target_col="Hazardous", top_n=3):
  df_train_numeric = df_train.select_dtypes(include=['number']).select_dtypes(exclude=['bool'])

  # If target column is not in numerical form
  if target_col not in df_train_numeric.columns:
      raise ValueError(f"Target column '{target_col}' must be numeric and present in the dataset.")

  corr_values = df_train_numeric.corr()[target_col].abs().sort_values(ascending=False)
  selected_features = corr_values.drop(index=target_col).head(top_n).index.tolist()

  # print(f"Selected features based on correlation with '{target_col}': {selected_features}")

  return df_train[selected_features], df_test[selected_features], df_valid[selected_features]

In [12]:
def perform_pca(df_train, df_test, df_valid, n_components=3):
  pca = PCA(n_components=n_components)

  df_train_numeric = df_train.select_dtypes(include=['number'])
  df_test_numeric = df_test.select_dtypes(include=['number'])
  df_valid_numeric = df_valid.select_dtypes(include=['number'])

  pca.fit(df_train_numeric)
  df_train_pca = pca.transform(df_train_numeric)
  df_test_pca = pca.transform(df_test_numeric)
  df_valid_pca = pca.transform(df_valid_numeric)

  pca_columns = [f'PC{i+1}' for i in range(n_components)]
  df_train_pca = pd.DataFrame(df_train_pca, columns=pca_columns)
  df_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
  df_valid_pca = pd.DataFrame(df_valid_pca, columns=pca_columns)

  return df_train_pca, df_test_pca, df_valid_pca

In [13]:
def numeric_conversion(df_orig, one_hot_encode_month=False):
  df = df_orig.copy()

  # Dropping 'Equinox' and 'Orbiting Body'
  df.drop(columns=['Equinox', 'Orbiting Body'], errors='ignore', inplace=True)

  # Converting 'Close Approach Date' to datetime
  df['Close Approach Date'] = pd.to_datetime(df['Close Approach Date'])
  df['Close Approach Year'] = df['Close Approach Date'].dt.year
  df['Close Approach Month'] = df['Close Approach Date'].dt.month

  # Converting 'Orbit Determination Date' to datetime
  df['Orbit Determination Date'] = pd.to_datetime(df['Orbit Determination Date'])
  df['Orbit Determination Year'] = df['Orbit Determination Date'].dt.year
  df['Orbit Determination Month'] = df['Orbit Determination Date'].dt.month


  # Converting 'Epoch Date Close Approach' to datetime
  df['Converted Date'] = df['Epoch Date Close Approach'].apply(
    lambda x: datetime.utcfromtimestamp(x / 1000) if pd.notnull(x) else None
  )

  df['Epoch Close Approach Year'] = df['Converted Date'].dt.year
  df['Epoch Close Approach Month'] = df['Converted Date'].dt.month


  # Encoding 'Hazardous' column
  df['Hazardous'] = df['Hazardous'].astype(int)

  # One-Hot Encoding for months (if needed)
  if one_hot_encode_month:
    df = pd.get_dummies(df, columns=['Close Approach Month', 'Epoch Close Approach Month', 'Orbit Determination Month'], prefix=['Close Approach Month no.', 'Epoch Close Approach Month no.', 'Orbit Determination Month no.'], dtype=int)

  df = df.drop(columns=["Converted Date", "Close Approach Date", "Epoch Date Close Approach", "Orbit Determination Date"])
  return df

In [14]:
def data_preprocessing(df):
  encoded = numeric_conversion(df)
  df_train, df_temp = train_test_split(encoded, test_size=0.4, random_state=42)
  df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

  no_outliers_train, no_outliers_valid, no_outliers_test = remove_outliers_zscore(df_train, df_valid, df_test)
  no_highly_correlated_train, no_highly_correlated_valid, no_highly_correlated_test = remove_highly_correlated(no_outliers_train, no_outliers_valid, no_outliers_test)
  normal_train, normal_test, normal_valid = normalize_minmax(no_highly_correlated_train, no_highly_correlated_valid, no_highly_correlated_test)
  normal_train.reset_index(drop=True, inplace=True)
  normal_valid.reset_index(drop=True, inplace=True)
  normal_test.reset_index(drop=True, inplace=True)
  return normal_train, normal_valid, normal_test